In [ ]:
%matplotlib inline

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne_bids import (BIDSPath, read_raw_bids)
import os, ccs_eeg_utils

bids_root = os.path.join('data', 'P3')
subject_id = '002'


bids_path = BIDSPath(subject=subject_id, session='P3', task='P3',
                     datatype='eeg', suffix='eeg', root=bids_root)

raw = read_raw_bids(bids_path)
ccs_eeg_utils.read_annotations_core(bids_path,raw)
raw.load_data()
raw.filter(0.5,50, fir_design='firwin')

In [ ]:
raw.set_montage('standard_1020', match_case=False)

In [ ]:
ica = mne.preprocessing.ICA(method="fastica")
ica.fit(raw,verbose=True)

In [ ]:
evts,evts_dict = mne.events_from_annotations(raw)
wanted_keys = [e for e in evts_dict.keys() if "response" in e]
evts_dict_stim=dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)

In [ ]:
epochs = mne.Epochs(raw,evts,evts_dict_stim,tmin=-0.1,tmax=1)
epochs.average().plot(picks='Oz')

In [ ]:
epochs = mne.Epochs(ica.get_sources(raw),evts,evts_dict_stim,tmin=-0.1,tmax=1)
epochs.average(picks=).plot()

In [ ]:
reconst_raw = raw.copy()
ica.apply(reconst_raw,exclude=[1,8,9])

#raw.plot()
#reconst_raw.plot()  
ica.plot_overlay(raw,exclude=[1,8,9])